In [ ]:
import importlib
import pkgutil
import shutil
from pathlib import Path
from typing import *

In [ ]:
MODULE = "captn"

client_module = importlib.import_module(f"{MODULE}")
dir(client_module)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__']

In [ ]:
def list_submodules(package_name: str) -> List[str]:
    def _list_submodules(list_name, package_name):
        for loader, module_name, is_pkg in pkgutil.walk_packages(
            package_name.__path__, package_name.__name__ + "."
        ):
            list_name.append(module_name)
            module_name = __import__(module_name, fromlist="dummylist")
            if is_pkg:
                _list_submodules(list_name, module_name)

    all_modules = []
    _list_submodules(all_modules, package_name)

    return all_modules


client_modules = list_submodules(client_module)
client_modules

[INFO] captn.testing.activate_by_import: Testing environment activated.


['captn._nbdev',
 'captn.cli',
 'captn.cli.cli',
 'captn.cli.db',
 'captn.cli.ds',
 'captn.cli.token',
 'captn.cli.version',
 'captn.cli.cli',
 'captn.cli.db',
 'captn.cli.ds',
 'captn.cli.token',
 'captn.cli.version',
 'captn.client',
 'captn.components',
 'captn.components.client',
 'captn.components.client',
 'captn.helper',
 'captn.logger',
 'captn.testing',
 'captn.testing.activate_by_import',
 'captn.testing.activate_by_import']

In [ ]:
def get_exported_symbols(submodules: List[str]) -> Dict[str, List[str]]:
    modules = {name: __import__(name, fromlist="dummylist") for name in submodules}
    return {
        name: module.__all__
        for name, module in modules.items()
        if hasattr(module, "__all__")
    }


symbols = get_exported_symbols(client_modules)
symbols

{'captn._nbdev': ['index', 'modules', 'custom_doc_links', 'git_url'],
 'captn.cli.cli': ['app'],
 'captn.cli.db': ['app'],
 'captn.cli.ds': ['app'],
 'captn.cli.token': ['logger'],
 'captn.cli.version': ['logger'],
 'captn.client': ['Client', 'DataBlob', 'DataSource'],
 'captn.components.client': ['Client'],
 'captn.helper': ['ensure_is_instance',
  'get_base_url',
  'post_data',
  'get_data',
  'delete_data'],
 'captn.logger': ['supress_timestamps',
  'get_logger',
  'should_supress_timestamps',
  'set_level'],
 'captn.testing.activate_by_import': ['activated']}

In [ ]:
def filter_symbols(
    symbols: Dict[str, List[str]], module_name: str
) -> Dict[str, List[str]]:
    return {k: v for k, v in symbols.items() if k == module_name}


filter_symbols(symbols, f"{MODULE}.client")

{'captn.client': ['Client', 'DataBlob', 'DataSource']}

In [ ]:
def write_md_files(fname: Path, s: str, m: str) -> None:
    with open(fname, mode="w") as f:
        print(f"Writing {fname}")
        f.write(f"#{s}\n::: {m}.{s}\n")

In [ ]:
def get_path(m: str, docs_path: Path) -> Tuple[str, Path]:
    parts = m.split(".")[-1:]
    path_link = "API/" + "/".join(parts)
    path = docs_path / path_link

    return path, path_link

In [ ]:
def append_api_nav(summary: str, symbols: dict, docs_path: str, v: str) -> str:
    symbols = filter_symbols(symbols, v)

    for m, sx in symbols.items():
        path, path_link = get_path(m, docs_path)
        summary = summary + f"    - {m}\n"
        path.mkdir(exist_ok=True, parents=True)

        for s in sx:
            summary = summary + f"        - [{s}]({path_link}/{s}.md)\n"
            fname = path / (s + ".md")
            write_md_files(fname, s, m)

    return summary

In [ ]:
def append_cli_nav(
    summary: str, symbols: dict, docs_path: Path, v: str, ignore_list: list
) -> str:
    keys = []

    path, path_link = get_path(v, docs_path)

    for m, sx in symbols.items():
        if f"{v}." in m:
            sm = m.split(".")[2]
            if sm not in ignore_list:
                if "_" in sm:
                    sm = sm.replace("_", "-")
                keys.append(sm)
    for sm in keys:
        #         folder_path = path / (sm)
        #         folder_path.mkdir(exist_ok=True, parents=True)

        summary = summary + f"    - [{sm}]({path_link}/{sm}/app.md)\n"

        fname = path / (sm + "/app.md")

    return summary

In [ ]:
_summary = """
- [Home](index.md)
- [Tutorial](Tutorial.md)
"""


def build_api(module, summary: str = _summary, docs_path: str = "../docs"):
    api_modules = {"API": f"{MODULE}.client"}

    cli_modules = {"CLI": f"{MODULE}.cli"}
    ignore_list = ["cli", "logger", "helper"]

    submodules = list_submodules(module)
    symbols = get_exported_symbols(submodules)

    docs_path = Path(docs_path)
    assert docs_path.exists()
    shutil.rmtree(docs_path / "API", ignore_errors=True)

    for k, v in api_modules.items():
        summary = summary + f"- {k}\n"
        summary = append_api_nav(summary, symbols, docs_path, v)

    for k, v in cli_modules.items():
        summary = summary + f"- {k}\n"
        summary = append_cli_nav(summary, symbols, docs_path, v, ignore_list)

    summary = summary + "- [Releases](RELEASE.md)"

    with open(docs_path / "SUMMARY.md", mode="w") as f:
        f.write(summary)

    return summary


print(build_api(client_module))

Writing ../docs/API/client/Client.md
Writing ../docs/API/client/DataBlob.md
Writing ../docs/API/client/DataSource.md

- [Home](index.md)
- [Tutorial](Tutorial.md)
- API
    - captn.client
        - [Client](API/client/Client.md)
        - [DataBlob](API/client/DataBlob.md)
        - [DataSource](API/client/DataSource.md)
- CLI
    - [db](API/cli/db/app.md)
    - [ds](API/cli/ds/app.md)
    - [token](API/cli/token/app.md)
    - [version](API/cli/version/app.md)
- [Releases](RELEASE.md)
